<a href="https://colab.research.google.com/github/CER-REC/esa-data-bank_banque-donnees-ees/blob/master/Codes/Section_05_VEC_Labeling_for_Tables/02_labeling_VCs_with_zero_shot_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

In [2]:
import pandas as pd
from transformers import pipeline
from bs4 import BeautifulSoup
import nltk
from nltk.stem.porter import *

nltk.download('stopwords', quiet=True)

True

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
df_joined = pd.read_csv('/content/drive/MyDrive/data/esa_index_files/esa_index_with_table_text_no_labels.csv', index_col=0)

df_joined['csvText'] = df_joined['csvText'].replace(' +', ' ', regex=True)

In [5]:
df_joined.head()

,Index,Title,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,Data ID,PDF Download URL,Application Type (NEB Act),Pipeline Location,Hearing order,Consultant Name,Pipeline Status,Regulatory Instrument(s),Application URL,Decision URL,ESA Section(s),ESA Section(s) Index,ESA Section(s) Topics,CSV Download URL,PDF Page Number,PDF Page Count,PDF Size,PDF Outline,Download folder name,Zipped Project Link,Missing CSV,CSV Filename,csvFileName,csvText
0,9134,TABLE 3 SUMMARY OF AQUATICS FIELD WORK AND ABO...,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Co...",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrth...,14,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-3-summary-of-aquatics-field-wor...,1059614_14_lattice-v_1.csv,Survey Date Aboriginal Communities Detail Jul...
1,9135,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Co...",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrth...,17,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_17_lattice-v_1.csv,Field Site No Name Legal Location UTM Coordin...
2,9136,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Co...",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrth...,18,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_18_lattice-v_1.csv,Field Site No Name Legal Location UTM Coordin...
3,9137,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Co...",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrth...,19,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_19_lattice-v_1.csv,Field Site No Name Legal Location UTM Coordin...
4,9138,TABLE 4 SUMMARY OF WATERCOURSE CROSS

In [6]:
# Explicitly ask for tensor allocation on CUDA device :0
classifier = pipeline("zero-shot-classification", device=0)

In [7]:
df_joined.iloc[100]['csvText']

' Full timeEmployees Part time VolunteerEmployees Equipment ServicesProvided Areas CommunitiesServed Hudson sHope FireDepartment 1 fire chief 25 firefighters Not specified Firesuppression publiceducation fireprevention andinspectionservices City of Hudson s Hopeand surrounding area MoberlyLake FireDepartment 1 fire chief Severalfirefighters notspecified Not specified Not specified Moberly Lake area West Moberly FirstNation and SaulteauFirst Nation SOURCE Modified from PRRD 2013a '

In [28]:
sequences = []

for x in range(len(df_joined)):
  sequences.append(df_joined.iloc[x]['csvText'][1:300])

In [13]:
len(sequences)

28891

In [14]:
candidate_labels = """Physical and meteorological environment,Soil and soil productivity,Vegetation,Water quality and quantity,Fish and fish habitat,Wetlands,Wildlife and wildlife habitat,Species at Risk or Species of Special Status and related habitat,Greenhouse gas (GHG) emissions and climate change,Air emissions,Acoustic environment,Electromagnetism and Corona Discharge,Human occupancy and resource use,Heritage resources,Navigation and navigation safety,Traditional land and resource use,Social and cultural well-being,Human health and aesthetics,Infrastructure""".split(',')
hypothesis_template = "The data from this table is about {}."
candidate_labels

['Physical and meteorological environment',
 'Soil and soil productivity',
 'Vegetation',
 'Water quality and quantity',
 'Fish and fish habitat',
 'Wetlands',
 'Wildlife and wildlife habitat',
 'Species at Risk or Species of Special Status and related habitat',
 'Greenhouse gas (GHG) emissions and climate change',
 'Air emissions',
 'Acoustic environment',
 'Electromagnetism and Corona Discharge',
 'Human occupancy and resource use',
 'Heritage resources',
 'Navigation and navigation safety',
 'Traditional land and resource use',
 'Social and cultural well-being',
 'Human health and aesthetics',
 'Infrastructure']

In [17]:
from itertools import zip_longest
 
def grouper(iterable_obj, count, fillvalue=None):
    args = [iter(iterable_obj)] * count
    return zip_longest(*args, fillvalue=fillvalue)

output = [] 
 
for sequence_batch in grouper(sequences, 10, ""):
    output.append(classifier(sequence_batch, candidate_labels, hypothesis_template=hypothesis_template, multi_label=True))

In [29]:
output[0]

[{'labels': ['Traditional land and resource use',
   'Water quality and quantity',
   'Social and cultural well-being',
   'Human occupancy and resource use',
   'Fish and fish habitat',
   'Heritage resources',
   'Navigation and navigation safety',
   'Vegetation',
   'Wetlands',
   'Physical and meteorological environment',
   'Soil and soil productivity',
   'Infrastructure',
   'Wildlife and wildlife habitat',
   'Species at Risk or Species of Special Status and related habitat',
   'Human health and aesthetics',
   'Electromagnetism and Corona Discharge',
   'Acoustic environment',
   'Air emissions',
   'Greenhouse gas (GHG) emissions and climate change'],
  'scores': [0.6877259612083435,
   0.649422287940979,
   0.4630372226238251,
   0.30576542019844055,
   0.08918734639883041,
   0.03485028073191643,
   0.03348839655518532,
   0.02901163510978222,
   0.017860960215330124,
   0.015356151387095451,
   0.012270933017134666,
   0.009136791341006756,
   0.008778736926615238,
   0.

In [23]:
# Since we have a list of lists for the outputs (because inference was done in batches), we need to flatten the lists together into one.
flatOutput = [item for elem in output for item in elem]

In [27]:
import pickle 
with open('zero_shot_vcs.pkl', 'wb') as f:
  pickle.dump(flatOutput, f)

In [20]:
df_output = pd.DataFrame(output)
df_output

{'labels': ['Traditional land and resource use',
  'Water quality and quantity',
  'Social and cultural well-being',
  'Human occupancy and resource use',
  'Fish and fish habitat',
  'Heritage resources',
  'Navigation and navigation safety',
  'Vegetation',
  'Wetlands',
  'Physical and meteorological environment',
  'Soil and soil productivity',
  'Infrastructure',
  'Wildlife and wildlife habitat',
  'Species at Risk or Species of Special Status and related habitat',
  'Human health and aesthetics',
  'Electromagnetism and Corona Discharge',
  'Acoustic environment',
  'Air emissions',
  'Greenhouse gas (GHG) emissions and climate change'],
 'scores': [0.6877259612083435,
  0.649422287940979,
  0.4630372226238251,
  0.30576542019844055,
  0.08918734639883041,
  0.03485028073191643,
  0.03348839655518532,
  0.02901163510978222,
  0.017860960215330124,
  0.015356151387095451,
  0.012270933017134666,
  0.009136791341006756,
  0.008778736926615238,
  0.006375862751156092,
  0.004991511

In [ ]:
df_output[df_output.scores > 0.8]